In [1]:
from PIL import Image
import cv2
import os
from tqdm import tqdm
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

In [8]:
class ImageProcessor:
    
    def path_sequence(self,root,*directories):
        seq = root
        for directory in directories:
            seq += f"/{directory}"
        return seq
    
    def get_screen_resolution(self):
        import tkinter
        root = tkinter.Tk()
        width = root.winfo_screenwidth()
        height = root.winfo_screenheight()
        return width,height
    
    def __init__(self):
        self.root = "."
        self.images_file_path = self.path_sequence(self.root,"test_images")
        self.video_file_path = self.path_sequence(self.root,"test_videos")

        
        print("Video üzerinde test:0\nGörüntüler üzerinde test:1\n")
        d = input(">")
        if d == "0":
            self.test_on_video()
        elif d == "1":
            self.test_on_images()
        else:
            print("Bilinmeyen Girdi")
    
    def predict(self,image):
        blur1 = cv2.blur(image,(10,10))
        uai_diff = self.find_UAI(blur1)
        blur2 =cv2.medianBlur(uai_diff,3)

        #circles = self.find_circles(uai_diff)
        #drawn_image = self.draw_circles(image,circles)
        return blur2
        #return drawn_image
        
    def draw_circles(self,image,circles):
        circles = np.round(circles[0, :]).astype("int")
        cv2.circle(image, center=(circles[0, 0], circles[0, 1]), radius=circles[0, 2], color=(0, 255, 0), thickness=5)
        return image
    
    def find_circles(self,image):

        circles = cv2.HoughCircles(blur,cv2.HOUGH_GRADIENT,1,20,param1=50,param2=30,minRadius=150,maxRadius=500)
        return circles  # [x,y,radius]

    def find_UAP(self):
        pass
    
    def find_UAI(self,image): #[0,0,255] bgr
        #hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Hedef renk aralığı
        lower_red = np.array([100, 100, 50])
        upper_red = np.array([255, 255, 255])
        mask = cv2.inRange(image, lower_red, upper_red)
        diff = cv2.bitwise_and(image, image, mask = mask)
        return diff
    

    def test_on_images(self):
        screen_w,screen_h = self.get_screen_resolution()

        images_name_list = os.listdir(self.images_file_path)
        print(len(images_name_list))

        for im_name in tqdm(images_name_list):
            
            cv2.namedWindow(im_name)
            cv2.moveWindow(im_name, 20,20) 
            im = cv2.imread(self.path_sequence(self.root,"test_images",im_name))

            out_im = self.predict(im)

            out_im = cv2.resize(out_im,(int(screen_w*0.6),int(screen_h*0.45)))
            im = cv2.resize(im,(int(screen_w*0.6),int(screen_h*0.45)))
            out_im = np.concatenate((out_im,im),axis=0)
            cv2.imshow(im_name,out_im)
            key = cv2.waitKey(0)
            if key == ord("q"):
                cv2.destroyAllWindows()
                break
            elif key == ord(" "):
                cv2.destroyAllWindows()
                continue
      
    
    def test_on_video(self):
        screen_w,screen_h = self.get_screen_resolution()
        
        videos_list = os.listdir(self.video_file_path)
        for video_name in tqdm(videos_list):
            cap = cv2.VideoCapture(self.path_sequence(self.video_file_path,video_name))
                         
            cv2.namedWindow(video_name)
            cv2.moveWindow(video_name, 20,20) 
            while True:
                ret,frame = cap.read()
                frame_ = self.predict(frame)
                frame_ = cv2.resize(frame_,(int(screen_w*0.6),int(screen_h*0.45)))
                frame = cv2.resize(frame,(int(screen_w*0.6),int(screen_h*0.45)))
                frame_ = np.concatenate((frame_,frame),axis=0)
                cv2.imshow(video_name,frame_)
            
                if cv2.waitKey(1) & 0xFF == ord('q'): 
                    break
                if ret == False:
                    break
            cap.release()
            cv2.destroyAllWindows()
        
    def show_on_console(self,*frames):
        fig = plt.figure(figsize=(10,5))
        num_rows = 2
        num_columns = 3    

        for idx,frame in enumerate(frames): 
            fig.add_subplot(num_rows,num_columns,idx+1)
            plt.imshow(frame)
            plt.axis("off")
            plt.title(str(idx))


In [9]:
ImageProcessor()

Video üzerinde test:0
Görüntüler üzerinde test:1

>1
13


 31%|█████████████████████████▌                                                         | 4/13 [00:16<00:38,  4.23s/it]
